<a href="https://colab.research.google.com/github/navas017/Grupo40-Practica1/blob/main/Practica1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **IMPORTACIÓN DE LIBRERÍAS** 

In [2]:
import pandas as pd
import numpy as np
import urllib.request
import bz2
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import PredefinedSplit
import matplotlib.pyplot as plt
import math
import time

# **ANÁLISIS EXPLORATORIO DE DATOS (EDA)**

# **PROCESAMIENTO DE DATOS**

Lectura de archivos

In [3]:
# Aprendizaje
disp_df = pd.read_csv("/content/drive/MyDrive/Aprendizaje Automático/Datos/disp_st40ns1.txt.bz2",
 compression="bz2",
 index_col=0)

# Test
comp_df = pd.read_csv("/content/drive/MyDrive/Aprendizaje Automático/Datos/comp_st40ns1.txt.bz2",
 compression="bz2",
 index_col=0)

FileNotFoundError: ignored

Normalización de valores

In [ ]:
# Seleccionar todas las columnas a normalizar
cols_to_norm_disp = disp_df.columns
cols_to_norm_comp = comp_df.columns

# Maximos y minimos para cada columna
dispMaxCol = disp_df[cols_to_norm_disp].max()
dispMinCol = disp_df[cols_to_norm_disp].min()

compMaxCol = comp_df[cols_to_norm_comp].max()
compMinCol = comp_df[cols_to_norm_comp].min()

# Escalar los valores de las columnas en el rango [0, 1]
disp_norm = (disp_df[cols_to_norm_disp] - dispMinCol) / (dispMaxCol - dispMinCol)
comp_norm = (comp_df[cols_to_norm_comp] - compMinCol) / (compMaxCol - compMinCol)

# Desnormalizar valor
disp_desnorm = disp_norm[cols_to_norm_disp] * (dispMaxCol - dispMinCol) + dispMinCol
comp_desnorm = comp_norm[cols_to_norm_comp] * (compMaxCol - compMinCol) + compMinCol

Partición de datos en conjuntos de entrenamiento y test

In [ ]:
# Conjunto de datos no normalizados
train = disp_df.iloc[:3650]
test = disp_df.iloc[3650:]

# Tamaños de los conjuntos de datos
n_train = train.shape[0]
n_test = test.shape[0]

# Conjunto de datos normalizados
train_norm = disp_norm.iloc[:3650]
test_norm = disp_norm.iloc[3650:]

Predefined Split para optimización de parámetros

In [ ]:
selector = [-1] * n_train + [0] * n_test
cv = PredefinedSplit(selector)
split = cv.get_n_splits(disp_df)

Randomización de las filas en los ficheros de train y test y valores de entrenamiento

In [ ]:
# Para la randomizacion de filas se usa la semilla 100429142
train_random = train.sample(frac=1, random_state=100429142).reset_index(drop=True)
test_random = test.sample(frac=1, random_state=100429142).reset_index(drop=True)

# Randomizacion de conjuntos normalizados
train_random_norm = train_norm.sample(frac=1, random_state=100429142).reset_index(drop=True)
test_random_norm = test_norm.sample(frac=1, random_state=100429142).reset_index(drop=True)

# Separa las variables independientes (X) de la variable dependiente (y)
X_train = train_random.iloc[:, :-1]
Y_train = train_random.iloc[:, -1]

# Variables de conjunto de datos normalizado
X_train_norm = train_random_norm.iloc[:, :-1]
Y_train_norm = train_random_norm.iloc[:, -1]

# **MODELOS BÁSICOS**

## **OMISIÓN DE HIPERPARÁMETROS**

Modelo *KNN* con omisión de hiperparámetros

In [ ]:
# USA LOS VALORES NORMALIZADOS
# EXPLICAR PORQUE SE NORMALIZA CON EL KNN
# TIENE 5 NEIGHBORS POR DEFECTO

# Crear el modelo con los parámetros por defecto
knn = KNeighborsRegressor()

# Iniciar la medición del tiempo
start_time = time.time()

# Entrena el modelo con el conjunto de entrenamiento
knn.fit(X_train_norm, Y_train_norm)

# Finalizar la medición del tiempo y calculo del tiempo
end_time = time.time()
knn_training_time = round((end_time - start_time) * 1000, 6)

# Realiza predicciones en el conjunto de entrenamiento
y_pred_train = knn.predict(X_train_norm)

# Calcula el MAE y el RMSE
knn_mae = mean_absolute_error(Y_train_norm, y_pred_train)
knn_rmse = math.sqrt(mean_squared_error(Y_train_norm, y_pred_train))

Modelo Árbol de decisión con omisión de hiperparámetros

In [ ]:
# EXPLICAR EL SOBREAPRENDIZAJE SI NO SE LIMITA LA MAX_DEPTH
# ACABA CON 2^N HOJAS (CADA HOJA ES UN VALOR DE SALIDA)

# Crear el modelo con los parámetros por defecto
tree = DecisionTreeRegressor()

# Iniciar la medición del tiempo
start_time = time.time()

# Entrenar el modelo
tree.fit(X_train, Y_train)

# Finalizar la medición del tiempo y calculo del tiempo
end_time = time.time()
decision_training_time = round((end_time - start_time) * 1000, 6)

# Realizar predicciones en el conjunto de entrenamiento
Y_pred = tree.predict(X_train)

# Calcular el MAE y el RMSE
decision_mae = mean_absolute_error(Y_train, Y_pred)
decision_rmse = math.sqrt(mean_squared_error(Y_train, Y_pred))

Modelo Árbol de regresión lineal con omisión de hiperparámetros

In [ ]:
# Crear modelo
modelo_rl = LinearRegression()

# Iniciar la medición del tiempo
start_time = time.time()

# Entrenar modelo
modelo_rl.fit(X_train_norm, Y_train_norm)

# Finalizar la medición del tiempo y calculo del tiempo
end_time = time.time()
reg_training_time = round((end_time - start_time) * 1000, 6)

# Predecir valores para datos de test
y_pred = modelo_rl.predict(X_train_norm)

# Calcular el MAE y el RMSE
reg_mae = mean_absolute_error(Y_train_norm, y_pred)
reg_rmse = math.sqrt(mean_squared_error(Y_train_norm, y_pred))

## **COMPARATIVA MODELOS BÁSICOS**

Tabla de datos

In [ ]:
# Crear un diccionario con los valores de MAE, RMSE y tiempo de ejecución de cada modelo
data = {'KNN': [knn_mae, knn_rmse, knn_training_time],
        'Árbol de decisión': [decision_mae, decision_rmse, decision_training_time],
        'Regresión lineal': [reg_mae, reg_rmse, reg_training_time]}

# Crear un DataFrame de pandas con los valores del diccionario y especificar las etiquetas de filas y columnas
df = pd.DataFrame(data, index=['MAE', 'RMSE', 'Tiempo de entrenamiento (ms)'])

# Imprimir el DataFrame
df

Gráfica comparativa del MAE

In [ ]:
# Obtener los valores de MAE para cada modelo
mae_values = df.loc['MAE']

# Crear un gráfico de barras
fig, ax = plt.subplots()
ax.bar(mae_values.index, mae_values.values)

# Personalizar el gráfico
ax.set_title('MAE para los tres modelos')
ax.set_xlabel('Modelo')
ax.set_ylabel('MAE')
plt.show()

Gráfica comparativa del RMSE

In [ ]:
# Obtener los valores de RMSE para cada modelo
rmse_values = df.loc['RMSE']

# Crear un gráfico de barras
fig, ax = plt.subplots()
ax.bar(rmse_values.index, rmse_values.values)

# Personalizar el gráfico
ax.set_title('RMSE para los tres modelos')
ax.set_xlabel('Modelo')
ax.set_ylabel('RMSE')
plt.show()

Gráfica comparativa del Tiempo de entrenamiento

In [ ]:
# Obtener los valores de tiempo de entrenamiento para cada modelo
time_values = df.loc['Tiempo de entrenamiento (ms)']

# Crear un gráfico de barras
fig, ax = plt.subplots()
ax.bar(time_values.index, time_values.values)

# Personalizar el gráfico
ax.set_title('Tiempo de entrenamiento para los tres modelos')
ax.set_xlabel('Modelo')
ax.set_ylabel('Tiempo de entrenamiento (ms)')
plt.show()

## **OPTIMIZACIÓN DE HIPERPARÁMETROS**

Modelo KNN con hiperparámetros óptimos

In [ ]:
# PARA LA OPTIMIZACION DE PARAMETROS HAY QUE OBTENER EL CV USANDO LOS SPLITS (AG DEL PROFE)
knn = KNeighborsRegressor()

# Definir cuadrícula de valores de hiperparámetros a explorar
param_grid = {'n_neighbors': [7, 8, 9, 10, 11, 12, 15]}

# Definir métricas de evaluación a utilizar
scoring = { 'MAE': make_scorer(mean_absolute_error, greater_is_better=False) }
#scoring = {'MAE': make_scorer(mean_absolute_error, greater_is_better=False), 'RMSE': make_scorer(mean_squared_error, squared=False, greater_is_better=False)}

# Realizar búsqueda exhaustiva de los mejores valores de los hiperparámetros
grid_search = GridSearchCV(knn, param_grid=param_grid, scoring=make_scorer(mean_absolute_error, greater_is_better=False), cv=5)
grid_search.fit(X_train_norm, Y_train_norm)

# Imprimir mejores valores de los hiperparámetros y su rendimiento en términos de MAE y RMSE
print('Mejores valores de hiperparámetros:', grid_search.best_params_)
print('MAE con hiperparámetros optimizados:', grid_search.cv_results_['mean_test_MAE'][grid_search.best_index_] * -1)
print('RMSE con hiperparámetros optimizados:', grid_search.cv_results_['mean_test_RMSE'][grid_search.best_index_] * -1)

Modelo Árbol de Decisión con hiperparámetros óptimos

In [ ]:
# Definir el modelo
tree = DecisionTreeRegressor()

# Definir los parámetros a probar
param_grid = {'max_depth': [3, 5, 7, 9], 'min_samples_leaf': [1, 2, 4, 8]}

scoring = {'MAE': make_scorer(mean_absolute_error, greater_is_better=False), 'RMSE': make_scorer(mean_squared_error, squared=False, greater_is_better=False)}

# Definir la búsqueda por validación cruzada
grid_search = GridSearchCV(tree, param_grid, cv=5, refit='MAE', scoring=scoring) #'neg_mean_squared_error'

# Entrenar el modelo con los datos de entrenamiento
grid_search.fit(X_train_norm, Y_train_norm)

# Obtener los mejores parámetros y el mejor modelo
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Realizar predicciones en el conjunto de entrenamiento
Y_pred = best_model.predict(X_train_norm)

# Calcular el MAE y el RMSE
mae = mean_absolute_error(Y_train_norm, Y_pred)
rmse = math.sqrt(mean_squared_error(Y_train_norm, Y_pred))

# Imprimir los resultados
print("Mejores parámetros: ", best_params)
print("MAE: ", mae)
print("RMSE: ", rmse)

Modelo Árbol de Regresión Lineal con hiperparámetros óptimos

HACER LASSO Y RIDGE

# **MODELOS AVANZADOS**

## **OMISIÓN DE HIPERPARÁMETROS**

Modelo SVMs con omisión de hiperparámetros

In [ ]:
# Crear un clasificador SVM sin ajuste de hiperparámetros
clf = SVC(kernel='linear', C=1, random_state=0)

# Entrenar el clasificador en los datos de entrenamiento
clf.fit(X_train, Y_train)

# Hacer predicciones en los datos de entrenamiento
y_pred_train = clf.predict(X_train)

# Evaluar el rendimiento del clasificador en los datos de entrenamiento
accuracy_train = clf.score(X_train, Y_train)
print("Accuracy en los datos de entrenamiento:", accuracy_train)

Modelo Random Forests con omisión de hiperparámetros

In [ ]:
# Crear modelo con los parámetros por defecto
rf = RandomForestRegressor()

# Entrenar el modelo
rf.fit(X_train, Y_train)

# Realizar predicciones en el conjunto de entrenamiento
Y_pred = rf.predict(X_train)

# Calcular el MAE y el RMSE
mae = mean_absolute_error(Y_train, Y_pred)
rmse = math.sqrt(mean_squared_error(Y_train, Y_pred))

# Imprimir los resultados
print("MAE: {:.2f}".format(mae))
print("RMSE: {:.2f}".format(rmse))

## **OPTIMIZACIÓN DE HIPERPARÁMETROS**

Modelo SVMs con hiperparámetros óptimos

In [ ]:
# Definir los parámetros a optimizar
param_grid = {'C': [0.1, 1, 10], 'epsilon': [0.1, 0.2, 0.3], 'gamma': ['scale', 'auto']}

# Crear el modelo SVR
svm = SVR()

# Realizar la búsqueda de los mejores parámetros
svm_opt = GridSearchCV(svm, param_grid, scoring='neg_mean_absolute_error', cv=5)
svm_opt.fit(X_train, Y_train)

# Obtener los mejores parámetros
best_params = svm_opt.best_params_
print("Mejores parámetros:", best_params)

# Entrenar el modelo con los mejores parámetros
svm = SVR(**best_params)
svm.fit(X_train, Y_train)

# Realizar predicciones en el conjunto de entrenamiento
Y_pred = svm.predict(X_train)

# Calcular el MAE y el RMSE
mae = mean_absolute_error(Y_train, Y_pred)
rmse = math.sqrt(mean_squared_error(Y_train, Y_pred))

# Imprimir los resultados
print("MAE: {:.2f}".format(mae))
print("RMSE: {:.2f}".format(rmse))

Modelo Random Forests con hiperparámetros óptimos

In [ ]:
# Crear modelo de Random Forest
rf = RandomForestRegressor()

# Definir grid de hiperparámetros a evaluar
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10],
    'max_features': ['sqrt', 'log2']
}

# Crear objeto GridSearchCV para encontrar los mejores parámetros
grid_rf = GridSearchCV(rf, param_grid, cv=3, n_jobs=-1, verbose=1)

# Entrenar modelo con parámetros optimizados
grid_rf.fit(X_train, Y_train)

# Imprimir los mejores parámetros encontrados
print("Mejores parámetros: ", grid_rf.best_params_)

# Predecir valores para datos de test usando el modelo optimizado
Y_pred = grid_rf.predict(X_train)

# Calcular el MAE y el RMSE
mae = mean_absolute_error(Y_train, Y_pred)
rmse = math.sqrt(mean_squared_error(Y_train, Y_pred))

# Imprimir los resultados
print("MAE: {:.2f}".format(mae))
print("RMSE: {:.2f}".format(rmse))